<a href="https://colab.research.google.com/github/mmahnoo44/Twitter-sentiment-analysis/blob/main/Twitter_sentiment_analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# fetch api  data set
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 98% 79.0M/80.9M [00:03<00:00, 32.3MB/s]
100% 80.9M/80.9M [00:03<00:00, 24.6MB/s]


In [ ]:
# extracting the compressed dataset
from zipfile import ZipFile

dataset = '/content/sentiment140.zip'

# Use proper indentation
with ZipFile(dataset, 'r') as zip:
    # Use extractall() method, not extractorall()
    zip.extractall()
    print('Extracted')  # Adjusted indentation for the print statement


Extracted


install dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from  sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics  import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

loading data rom csv to pandas

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
#printing 1st 5 rows of datafram
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#naming column and reading dataset again
column_names=['target' , 'id' , 'date' , 'flag' , 'user' , 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',  names=column_names , encoding = 'ISO-8859-1')


In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#counting the nummof missing vales in dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
#checking the  distribution of target colummn
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

convert target 4 to 1

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)


In [ ]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0->negative Tweet
1->Positive Tweet

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Stemming**

stemming is the process of reducing a word to its root word

example:actor , actress  , acting = act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    # Initialize a PorterStemmer


    # Replace non-alphabetic characters with space
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)

    # Convert to lowercase and split into words
    stemmed_content = stemmed_content.lower()
    stemmed_words = word_tokenize(stemmed_content)

    # Remove stopwords and perform stemming
    stemmed_words = [port_stem.stem(word) for word in stemmed_words if word not in stopwords.words('english')]

    # Join stemmed words back into a string
    stemmed_content = ' '.join(stemmed_words)

    return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


separartinng theee data and label

In [ ]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming X and Y are your feature and label arrays/dataframes
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

# Now you can use X_train, X_test, Y_train, and Y_test in your machine learning model


In [ ]:
print(X.shape,  X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gon na find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [ ]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [ ]:
 #converting the textual data to numerical

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

  (0, 443064)	0.4484755317023172
  (0, 235043)	0.41996827700291095
  (0, 109305)	0.3753708587402299
  (0, 185192)	0.5277679060576009
  (0, 354541)	0.3588091611460021
  (0, 436711)	0.27259876264838384
  (1, 160635)	1.0
  (2, 288468)	0.165559978152293
  (2, 132310)	0.20010573505848944
  (2, 282873)	0.165740495217716
  (2, 150654)	0.18506405974027818
  (2, 178060)	0.1596736064437859
  (2, 409141)	0.14960586122844477
  (2, 266727)	0.23791347672998542
  (2, 443428)	0.33025302483687713
  (2, 77928)	0.30853680088497654
  (2, 433558)	0.32512419345325
  (2, 406397)	0.3166375845052528
  (2, 129410)	0.2867419530138451
  (2, 407299)	0.18451939024838399
  (2, 124483)	0.18661240251230476
  (2, 109305)	0.45280118482452514
  (3, 172420)	0.375557395676497
  (3, 411526)	0.27156001736034
  (3, 388624)	0.3950410772895712
  :	:
  (1279996, 390128)	0.22064742191076112
  (1279996, 434012)	0.2718945052332447
  (1279996, 318301)	0.21254698865277746
  (1279996, 237897)	0.2236567560099234
  (1279996, 291076)	0.1

In [ ]:
print(X_test)

  (0, 420982)	0.17915624523539803
  (0, 409141)	0.31430470598079707
  (0, 398904)	0.3491043873264267
  (0, 388346)	0.21985076072061738
  (0, 279080)	0.1782518010910344
  (0, 271014)	0.4535662391658828
  (0, 171377)	0.2805816206356073
  (0, 138163)	0.23688292264071403
  (0, 132363)	0.25525488955578596
  (0, 106068)	0.3655545001090455
  (0, 67827)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366201)	0.24595562404108307
  (1, 348133)	0.4739279595416274
  (1, 256775)	0.28751585696559306
  (1, 217561)	0.40288153995289894
  (1, 145392)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400619)	0.4317732461913093
  (2, 256832)	0.2564939661498776
  (2, 183311)	0.5892069252021465
  (2, 89447)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123277)	0.4530341382559843
  (319995, 444932)	0.3211092817599261
  (319995, 420982)	0.22631428606830145
  (319995, 416255)	0.23816465111736276
  (319995, 3

logistic regression

In [ ]:
model = LogisticRegression(max_iter=1000)


In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('Accuracy score  on the training data:', training_data_accuracy)

Accuracy score  on the training data: 0.81015234375


In [57]:
# axccuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)


In [58]:
print('Accuracy score  on the training data:', test_data_accuracy)

Accuracy score  on the training data: 0.7779875


model accuracy 78%

In [60]:
import pickle

filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [61]:
#loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav' , 'rb'))

In [64]:
X_new = X_test[200]
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')

else:
  print('Positive Tweet')

1
[1]
Positive Tweet


In [65]:
X_new = X_test[3]
print(Y_test[3])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')

else:
  print('Positive Tweet')

0
[0]
Negative Tweet
